# Equal-Weight S&P 500 Index Fund

## Introduction & Library Imports

The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. It has more than US$250 billion of assets under management.

The goal of this section of the course is to create a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.

## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np
import pandas as pd
import requests 
import xlsxwriter
import math

## Importing Our List of Stocks

The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis.

Paying for access to the index provider's API is outside of the scope of this course. 

There's a static version of the S&P 500 constituents available here. [Click this link to download them now](https://drive.google.com/file/d/1ZJSpbY69DVckVZlO9cC6KkgfSufybcHN/view?usp=sharing). Move this file into the `starter-files` folder so it can be accessed by other files in that directory.

Now it's time to import these stocks to our Jupyter Notebook file.

In [28]:
# Connect to a finiacial data API or Index provider for up-to-date info
stocks = pd.read_csv('sp_500_stocks.csv')
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


## Acquiring an API Token

Now it's time to import our IEX Cloud API token. This is the data provider that we will be using throughout this course.

API tokens (and other sensitive information) should be stored in a `secrets.py` file that doesn't get pushed to your local Git repository. We'll be using a sandbox API token in this course, which means that the data we'll use is randomly-generated and (more importantly) has no cost associated with it.

[Click here](http://nickmccullum.com/algorithmic-trading-python/secrets.py) to download your `secrets.py` file. Move the file into the same directory as this Jupyter Notebook before proceeding.

In [29]:
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call

Now it's time to structure our API calls to IEX cloud. 

We need the following information from the API:

* Market capitalization for each stock
* Price of each stock



In [30]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json() # response object-> status code (404, 200)
#print(data.status_code)
print(data)

{'symbol': 'AAPL', 'companyName': 'Apple Inc', 'primaryExchange': 'EMTGQL SA/OAENKE)RB GCN ASDAL(TSL', 'calculationPrice': 'close', 'open': 135.72, 'openTime': 1629625192633, 'openSource': 'fiailcof', 'close': 134.61, 'closeTime': 1652152356782, 'closeSource': 'aiociffl', 'high': 136.12, 'highTime': 1686934043398, 'highSource': '1uarne mded i5telecp yi', 'low': 135.82, 'lowTime': 1682336367743, 'lowSource': '1mpre cdti5ey lnieaeu d', 'latestPrice': 137.8, 'latestSource': 'Close', 'latestTime': 'February 3, 2021', 'latestUpdate': 1666050920606, 'latestVolume': 92263161, 'iexRealtimePrice': 136.592, 'iexRealtimeSize': 2, 'iexLastUpdated': 1638340460911, 'delayedPrice': 138, 'delayedPriceTime': 1685178256421, 'oddLotDelayedPrice': 137, 'oddLotDelayedPriceTime': 1663497772260, 'extendedPrice': 137.23, 'extendedChange': 2.72, 'extendedChangePercent': 0.02077, 'extendedPriceTime': 1685397593527, 'previousClose': 135.95, 'previousVolume': 84622498, 'change': -1.09, 'changePercent': -0.00784, 

## Parsing Our API Call

The API call that we executed in the last code block contains all of the information required to build our equal-weight S&P 500 strategy. 

With that said, the data isn't in a proper format yet. We need to parse it first.

In [31]:
price = data['latestPrice']
market_cap = data['marketCap']

## Adding Our Stocks Data to a Pandas DataFrame

The next thing we need to do is add our stock's price and market capitalization to a pandas DataFrame. Think of a DataFrame like the Python version of a spreadsheet. It stores tabular data.

In [32]:
my_columns = [ 'Ticker', 'Stock Price', 'Market Capitalization', 'Number of Shares to Buy']
final_dataframe = pd.DataFrame(columns=my_columns)
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy


In [33]:
# Append data to data frame
final_dataframe.append(pd.Series([symbol,price,market_cap,'N/A'],index=my_columns),ignore_index=True)

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,AAPL,137.8,2361020604042,N/A


## Looping Through The Tickers in Our List of Stocks

Using the same logic that we outlined above, we can pull data for all S&P 500 stocks and store their data in the DataFrame using a `for` loop.

In [41]:
final_dataframe = pd.DataFrame(columns=my_columns)
for stock in stocks['Ticker']:
    api_url = f'https://sandbox.iexapis.com/stable/stock/{stock}/quote/?token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(api_url).json() # batch api request saves time
    final_dataframe = final_dataframe.append(
        pd.Series(
        [
            stock,
            data['latestPrice'],
            data['marketCap'],
            'N/A'
        ],
        index=my_columns),
    ignore_index=True
    )

In [42]:
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,124.50,38995367429,N/A
1,AAL,18.00,10863383836,N/A
2,AAP,157.22,10600043285,N/A
3,AAPL,137.01,2271554498189,N/A
4,ABBV,111.63,193127095079,N/A
...,...,...,...,...
500,YUM,107.74,32337828462,N/A
501,ZBH,163.46,33419763619,N/A
502,ZBRA,422.84,22093531635,N/A
503,ZION,47.11,7790614963,N/A


## Using Batch API Calls to Improve Performance

Batch API calls are one of the easiest ways to improve the performance of your code.

This is because HTTP requests are typically one of the slowest components of a script.

Also, API providers will often give you discounted rates for using batch API calls since they are easier for the API provider to respond to.

IEX Cloud limits their batch API calls to 100 tickers per request. Still, this reduces the number of API calls we'll make in this section from 500 to 5 - huge improvement! In this section, we'll split our list of stocks into groups of 100 and then make a batch API call for each group.

In [43]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst"""
    for i in range(0,len(lst),n):
        yield lst[i:i + n]

In [59]:
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
 #   print(symbol_strings[i])

final_dataframe = pd.DataFrame(columns=my_columns)
                               
for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    #print(data.status_code) check if call works
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
            pd.Series(
            [
                symbol,
                data[symbol]['quote']['latestPrice'],
                data[symbol]['quote']['marketCap'],
                'N/A'
            ],
            index=my_columns),
            ignore_index=True
        )
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,122.72,38291661878,N/A
1,AAL,17.80,11106466068,N/A
2,AAP,157.35,10679224341,N/A
3,AAPL,135.49,2338636276409,N/A
4,ABBV,109.09,189278478909,N/A
...,...,...,...,...
500,YUM,106.39,33037142053,N/A
501,ZBH,164.72,34048428160,N/A
502,ZBRA,417.15,22595614302,N/A
503,ZION,47.93,7850263121,N/A


## Calculating the Number of Shares to Buy

As you can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

We'll do that next.

In [64]:
portfolio_size = input('Enter the value of your portfolio:')

try:
    val = float(portfolio_size)
except ValueError:
    print('Please enter an integer')
    portfolio_size = input('Enter the value of your portfolio:')
    val = float(portfolio_size)

Enter the value of your portfolio:1000000


In [67]:
position_size = val/len(final_dataframe.index)
for i in range(0,len(final_dataframe.index)):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/final_dataframe.loc[i, 'Stock Price'])
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,122.72,38291661878,16
1,AAL,17.80,11106466068,111
2,AAP,157.35,10679224341,12
3,AAPL,135.49,2338636276409,14
4,ABBV,109.09,189278478909,18
...,...,...,...,...
500,YUM,106.39,33037142053,18
501,ZBH,164.72,34048428160,12
502,ZBRA,417.15,22595614302,4
503,ZION,47.93,7850263121,41


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

### Initializing our XlsxWriter Object

In [82]:
writer = pd.ExcelWriter('recommended_trades.xlsx', engine = 'xlsxwriter')
final_dataframe.to_excel(writer, 'Recommended Trades', index = False)

### Creating the Formats We'll Need For Our `.xlsx` File

Formats include colors, fonts, and also symbols like `%` and `$`. We'll need four main formats for our Excel document:
* String format for tickers
* \\$XX.XX format for stock prices
* \\$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

In [83]:
# color scheme
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
    {
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1 
    }
)

dollar_format = writer.book.add_format(
    {
        'num_format': '$0.00',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1 
    }
)

integer_format = writer.book.add_format(
    {
        'num_format': '0',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1 
    }
)

### Applying the Formats to the Columns of Our `.xlsx` File

We can use the `set_column` method applied to the `writer.sheets['Recommended Trades']` object to apply formats to specific columns of our spreadsheets.

Here's an example:

```python
writer.sheets['Recommended Trades'].set_column('B:B', #This tells the method to apply the format to column B
                     18, #This tells the method to apply a column width of 18 pixels
                     string_template #This applies the format 'string_template' to the column
                    )
```

In [84]:
# writer.sheets['Recommended Trades'].set_column('A:A',18,string_format)
# writer.sheets['Recommended Trades'].set_column('B:B',18,string_format)
# writer.sheets['Recommended Trades'].set_column('C:C',18,string_format)
# writer.sheets['Recommended Trades'].set_column('D:D',18,string_format)

# writer.sheets['Recommended Trades'].write('A1','Ticker',string_format)
# writer.sheets['Recommended Trades'].write('B1','Stock Price',dollar_format)
# writer.sheets['Recommended Trades'].write('C1','Market Capitalization',dollar_format)
# writer.sheets['Recommended Trades'].write('D1','Number of Shares to Buy',integer_format)

This code works, but it violates the software principle of "Don't Repeat Yourself". 

Let's simplify this by putting it in 2 loops:

In [85]:
column_formats = {
    'A': ['Ticker', string_format],
    'B': ['Stock Price', dollar_format],
    'C': ['Market Capitalization', dollar_format],
    'D': ['Number of Shares to Buy', integer_format]
}

# loop through every key
for col in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{col}:{col}',18,column_formats[col][1])
    writer.sheets['Recommended Trades'].write(f'{col}1',column_formats[col][0],column_formats[col][1])


## Saving Our Excel Output

Saving our Excel file is very easy:

In [86]:
writer.save()